# Processamento de Linguagem Natural - Trabalho Prático 1
### Thaís Ferreira da Silva - 2021092571

### Objetivo
O objetivo desse trabalho é entender e aplicar conceitos de word embeddings, analisando a qualidade dos modelos treinados em função dos parâmetros escolhidos para o seu treinamento.

### Atividades Principais
Para esse trabalho, utilizei o gensim no treinamento do modelo atraves da variação dos parâmetros do Word2Vec.

Dividi esse trabalho em 4 partes principais:

##### Funções auxiliares: 
Parte contendo todas os métodos implementados para as partes seguintes

##### Treinamento: 
Aqui defini os parametros de treinamento e realizei de fato o treinamento dos 54 modelos resultado da variação dos parâmetos: 
- tamanho do vetor de palavras
- tamanho da janela de contexto
- número de interações
- utiliza CBOW ou skip-gram. 

Realizei o treinamento 2 vezes com parâmetros diferentes e armazenei em 2 pastas separadas para futuras analises de desempenho.

##### Avaliação: 
Nessa parte utilizei a distância média do cosseno (metrica exigida pelo trabalho) e a acurácia do modelo para escolher o melhor modelo treinado de acordo com as predições sobre analogias.


### Import

In [2]:
# Imports do gensim - para word2vec
import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus

# Imports essenciais
from scipy.spatial.distance import cosine
import os
import random

### Funções Auxiliares

Optei por desenvolver 4 principais funções com os seguintes objetivos:

- Gerar as combinaçoes de hiperparametros: São 54 variações, onde 3 parâmetros possuem 3 variações, e o for fim temos a variação do skip-gram e CBOW

- Treinar e salvar os modelos: Os modelos são treinados com Word2Vac e para facilitar as analises após o treinamento de cada modelo, eles são salvos em 2 pastas diferentes

- Pegar as analogias: Aqui o arquivo de texto com as analogias são lidos de 4 em 4 palavras e armazenados em um vetor que posteriormente será utilizado para a previsão e avaliação dos modelos

- Avaliar o model: Na avaliação dos modelos, é necessário passar por todas as analogias, e realizar a operação vetorial e prever o 4° elemento do vetor. Dessa forma, ao final de todo o processamento temos os calculos das métricas de avaliação, sendo elas a distância de cosseno e a acurácia.

In [3]:
def generate_hyperparameter_combinations(param_grid):
    from itertools import product
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in product(*values)]
    return combinations


In [4]:
def train_and_save_model(corpus, params, output_dir):
    model_name = f"word2vec_vs{params['vector_size']}_win{params['window']}_sg{params['sg']}_ep{params['epochs']}"
    print(f"Treinando modelo: {model_name}")
    
    model = Word2Vec(
        sentences=corpus,
        vector_size=params['vector_size'],
        window=params['window'],
        sg=params['sg'],
        epochs=params['epochs']
    )
    
    model_path = os.path.join(output_dir, f"{model_name}.model")
    model.save(model_path)
    print(f"Modelo salvo: {model_path}")
    
    return model

In [5]:
def get_analogies(model, questions_words_path):
    final_analogies = []
    with open(questions_words_path, 'r') as f:
        for line in f:
            if line.startswith(':'):
                continue

            words = [word.lower().strip() for word in line.split()]

            final_words = [word for word in words if word in model.wv]
            if len(final_words) != 4:
                continue
            
            final_analogies.append(final_words)
    
    random.shuffle(final_analogies)
    return final_analogies

In [6]:
def evaluate_models(model, analogies):
    avg_distance = 0
    correct = 0
    count = 0
    accuracy = 0

    for analogy in analogies:
        if len(analogy) == 4:
            count += 1

            try:
                result_vector = model.wv[analogy[1]] - model.wv[analogy[0]] + model.wv[analogy[2]]

                predicted = model.wv.similar_by_vector(result_vector, topn=20, restrict_vocab=None)
                predicted_word = next((word for word, _ in predicted if word not in analogy[:3]), None)

                if predicted_word == analogy[3]:
                    correct += 1

                if predicted_word in model.wv:
                    cosine_distance = cosine(model.wv[analogy[3]], model.wv[predicted_word])
                    avg_distance += cosine_distance

            except KeyError as e:
                print(e)

    accuracy = correct / count if count > 0 else 0
    avg_distance = avg_distance / count if count > 0 else 0

    return accuracy, avg_distance

### Inicialização
Aqui definimos lemos o arquivo do courpus, e definimos as pastas onde os modelos serão armazenados futuramente

In [7]:
text8_path = './text8'
corpus = Text8Corpus(text8_path)

sentence = next(iter(corpus))
print(sentence[:15])

output_dir = './word2vec_models_v1'
os.makedirs(output_dir, exist_ok=True)

output_dir2 = './word2vec_models_v2'
os.makedirs(output_dir2, exist_ok=True)

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']


### Treinamento do modelo Word2Vec

Na parte do treinamento busquei testar algumas variáções que me dessem resultados interessantes, mas sem demorar muito tempo para treinar no meu notebook (entre 6 e 10 horas).

As variações cobrem:
- embeddings 50, 100, 200, 300
- janela de tamanho 2 a 7
- modelos CBOW e Skip-gram
- numero de épocas: 4, 5, 8, 10, 15, 16

In [19]:
# Hiperparâmetros para o GridSearch
param_grid = {
    'vector_size': [50, 100, 200],      # Tamanho do vetor de palavras
    'window': [3, 5, 7],                # Tamanho da janela de contexto
    'sg': [0, 1],                      # CBOW (0) ou Skip-gram (1)
    'epochs': [5, 10, 15],             # Número de iterações de treinamento
}

combinations = generate_hyperparameter_combinations(param_grid)

for i, params in enumerate(combinations):
    print(f"\nTreinando combinação {i+1}/{len(combinations)}: {params}")
    model = train_and_save_model(corpus, params, output_dir)


Treinando combinação 1/54: {'vector_size': 50, 'window': 3, 'sg': 0, 'epochs': 5}
Treinando modelo: word2vec_vs50_win3_sg0_ep5
Modelo salvo: ./word2vec_models/word2vec_vs50_win3_sg0_ep5.model

Treinando combinação 2/54: {'vector_size': 50, 'window': 3, 'sg': 0, 'epochs': 10}
Treinando modelo: word2vec_vs50_win3_sg0_ep10
Modelo salvo: ./word2vec_models/word2vec_vs50_win3_sg0_ep10.model

Treinando combinação 3/54: {'vector_size': 50, 'window': 3, 'sg': 0, 'epochs': 15}
Treinando modelo: word2vec_vs50_win3_sg0_ep15
Modelo salvo: ./word2vec_models/word2vec_vs50_win3_sg0_ep15.model

Treinando combinação 4/54: {'vector_size': 50, 'window': 3, 'sg': 1, 'epochs': 5}
Treinando modelo: word2vec_vs50_win3_sg1_ep5
Modelo salvo: ./word2vec_models/word2vec_vs50_win3_sg1_ep5.model

Treinando combinação 5/54: {'vector_size': 50, 'window': 3, 'sg': 1, 'epochs': 10}
Treinando modelo: word2vec_vs50_win3_sg1_ep10
Modelo salvo: ./word2vec_models/word2vec_vs50_win3_sg1_ep10.model

Treinando combinação 6/54

In [22]:
# Hiperparâmetros para o GridSearch
param_grid2 = {
    'vector_size': [100, 200, 300],      # Tamanho do vetor de palavras
    'window': [2, 4, 6],                # Tamanho da janela de contexto
    'sg': [0, 1],                      # CBOW (0) ou Skip-gram (1)
    'epochs': [4, 8, 16],             # Número de iterações de treinamento
}

combinations2 = generate_hyperparameter_combinations(param_grid2)

for j, params in enumerate(combinations2):
    print(f"\nTreinando combinação {j+1}/{len(combinations2)}: {params}")
    model2 = train_and_save_model(corpus, params, output_dir2)


Treinando combinação 1/54: {'vector_size': 100, 'window': 2, 'sg': 0, 'epochs': 4}
Treinando modelo: word2vec_vs100_win2_sg0_ep4
Modelo salvo: ./word2vec_models_v2/word2vec_vs100_win2_sg0_ep4.model

Treinando combinação 2/54: {'vector_size': 100, 'window': 2, 'sg': 0, 'epochs': 8}
Treinando modelo: word2vec_vs100_win2_sg0_ep8
Modelo salvo: ./word2vec_models_v2/word2vec_vs100_win2_sg0_ep8.model

Treinando combinação 3/54: {'vector_size': 100, 'window': 2, 'sg': 0, 'epochs': 16}
Treinando modelo: word2vec_vs100_win2_sg0_ep16
Modelo salvo: ./word2vec_models_v2/word2vec_vs100_win2_sg0_ep16.model

Treinando combinação 4/54: {'vector_size': 100, 'window': 2, 'sg': 1, 'epochs': 4}
Treinando modelo: word2vec_vs100_win2_sg1_ep4
Modelo salvo: ./word2vec_models_v2/word2vec_vs100_win2_sg1_ep4.model

Treinando combinação 5/54: {'vector_size': 100, 'window': 2, 'sg': 1, 'epochs': 8}
Treinando modelo: word2vec_vs100_win2_sg1_ep8
Modelo salvo: ./word2vec_models_v2/word2vec_vs100_win2_sg1_ep8.model

T

### Avaliação dos modelos - primeiro treinamento

Para a analise inicialmente testei utilizando a distância do cosseno, mas os resultados não foram satisfatórios, tendo em vista que na maior parte dos modelos apontados como bons por essa métrica, quase nunca acertavam realmente a palavra esperada na predição.

Por esse motivo, adicionei uma nova métrica aos testes, sendo ele a acurácia que é uma métrica de desempenho amaplamente utilizada e que busca medir a proporção de previsões corretas em relação ao total de previsões realizadas

Por fim, gerei um top 10 para cada uma das métricas, com o objetivo de visualizar melhor a infuência das métricas

In [8]:
questions_words_path = './questions-words.txt'

models = os.listdir(output_dir)
models = [model for model in models if model.endswith('.model')]

model_metrics = []

num_modelo = 0

for model_name in models:
    num_modelo +=1
    model = Word2Vec.load(os.path.join(output_dir, model_name))
    analogies = get_analogies(model, questions_words_path)
    accuracy, avg_distance = evaluate_models(model, analogies)
    #avg_distance -> quanto menor melhor
    print(f"Modelo: [{num_modelo}], vector_size: {model.vector_size}, window: {model.window}, sg: {model.sg}, epochs: {model.epochs} ---> Accuracy: {accuracy}, avg_distance: {avg_distance}")
    model_metrics.append({
        'model_name': model_name,
        'accuracy': accuracy,
        'avg_distance': avg_distance,
        'vector_size': model.vector_size,
        'window': model.window,
        'sg': model.sg,
        'epochs': model.epochs
    })

Modelo: [1], vector_size: 50, window: 3, sg: 0, epochs: 15 ---> Accuracy: 0.2002580355640321, avg_distance: 0.3681690385566878
Modelo: [2], vector_size: 50, window: 5, sg: 0, epochs: 10 ---> Accuracy: 0.18387838671677792, avg_distance: 0.3789738460952951
Modelo: [3], vector_size: 50, window: 7, sg: 1, epochs: 10 ---> Accuracy: 0.2486116564761317, avg_distance: 0.2615548641385404
Modelo: [4], vector_size: 50, window: 7, sg: 0, epochs: 5 ---> Accuracy: 0.14141470802714984, avg_distance: 0.3855906066564134
Modelo: [5], vector_size: 200, window: 3, sg: 0, epochs: 5 ---> Accuracy: 0.17613731979581534, avg_distance: 0.38990197765251505
Modelo: [6], vector_size: 100, window: 7, sg: 1, epochs: 10 ---> Accuracy: 0.3213664665956134, avg_distance: 0.29455277380952716
Modelo: [7], vector_size: 100, window: 3, sg: 0, epochs: 15 ---> Accuracy: 0.25730633309025636, avg_distance: 0.39207872305244157
Modelo: [8], vector_size: 200, window: 7, sg: 1, epochs: 10 ---> Accuracy: 0.3492455264486453, avg_dist

In [12]:
print(f"Melhores modelos de acordo com a distância de cosseno:")
top_10_models = sorted(model_metrics, key=lambda x: x['avg_distance'])[:10]
for model_info in top_10_models:
    print(f"Model: {model_info['model_name']}, vector_size: {model_info['vector_size']}, window: {model_info['window']}, sg: {model_info['sg']}, epochs: {model_info['epochs']} ---> Accuracy: {model_info['accuracy']}, avg_distance: {model_info['avg_distance']}")


print(f"\n\nMelhores modelos de acordo com a acurácia:")
top_10_models = sorted(model_metrics, key=lambda x: x['accuracy'], reverse=True)[:10]
for model_info in top_10_models:
    print(f"Model: {model_info['model_name']}, vector_size: {model_info['vector_size']}, window: {model_info['window']}, sg: {model_info['sg']}, epochs: {model_info['epochs']} ---> Accuracy: {model_info['accuracy']}, avg_distance: {model_info['avg_distance']}")

Melhores modelos de acordo com a distância de cosseno:
Model: word2vec_vs50_win5_sg1_ep5.model, vector_size: 50, window: 5, sg: 1, epochs: 5 ---> Accuracy: 0.2193863241151063, avg_distance: 0.23655692735678963
Model: word2vec_vs50_win3_sg1_ep5.model, vector_size: 50, window: 3, sg: 1, epochs: 5 ---> Accuracy: 0.19851910024120717, avg_distance: 0.23700236297514105
Model: word2vec_vs50_win7_sg1_ep5.model, vector_size: 50, window: 7, sg: 1, epochs: 5 ---> Accuracy: 0.22684691759690356, avg_distance: 0.24140134295815982
Model: word2vec_vs50_win3_sg1_ep10.model, vector_size: 50, window: 3, sg: 1, epochs: 10 ---> Accuracy: 0.22993212542772198, avg_distance: 0.25670031706865015
Model: word2vec_vs50_win5_sg1_ep10.model, vector_size: 50, window: 5, sg: 1, epochs: 10 ---> Accuracy: 0.24384360801032143, avg_distance: 0.25797210377310825
Model: word2vec_vs50_win7_sg1_ep10.model, vector_size: 50, window: 7, sg: 1, epochs: 10 ---> Accuracy: 0.2486116564761317, avg_distance: 0.2615548641385404
Model:

### Avaliação dos modelos - segundo treinamento

In [8]:
questions_words_path = './questions-words.txt'

models2 = os.listdir(output_dir2)
models2 = [model2 for model2 in models2 if model2.endswith('.model')]

model_metrics2 = []

num_modelo2 = 0

for model_name2 in models2:
    num_modelo2 +=1
    model2 = Word2Vec.load(os.path.join(output_dir2, model_name2))
    analogies2 = get_analogies(model2, questions_words_path)
    accuracy2, avg_distance2 = evaluate_models(model2, analogies2)
    #avg_distance -> quanto menor melhor
    print(f"Modelo: [{num_modelo2}], vector_size: {model2.vector_size}, window: {model2.window}, sg: {model2.sg}, epochs: {model2.epochs} ---> Accuracy: {accuracy2}, avg_distance: {avg_distance2}")
    model_metrics2.append({
        'model_name': model_name2,
        'accuracy': accuracy2,
        'avg_distance': avg_distance2,
        'vector_size': model2.vector_size,
        'window': model2.window,
        'sg': model2.sg,
        'epochs': model2.epochs
    })

Modelo: [1], vector_size: 200, window: 6, sg: 1, epochs: 8 ---> Accuracy: 0.32203960284961014, avg_distance: 0.34091254308838276
Modelo: [2], vector_size: 100, window: 6, sg: 1, epochs: 16 ---> Accuracy: 0.3343804341728838, avg_distance: 0.3077461235099991
Modelo: [3], vector_size: 200, window: 6, sg: 1, epochs: 4 ---> Accuracy: 0.2746395916306726, avg_distance: 0.30582096324294855
Modelo: [4], vector_size: 300, window: 6, sg: 1, epochs: 8 ---> Accuracy: 0.30341616648903347, avg_distance: 0.38158379117528807
Modelo: [5], vector_size: 100, window: 2, sg: 1, epochs: 8 ---> Accuracy: 0.2559039658944298, avg_distance: 0.2915923016449943
Modelo: [6], vector_size: 100, window: 4, sg: 0, epochs: 8 ---> Accuracy: 0.2118135412576429, avg_distance: 0.3942496256899785
Modelo: [7], vector_size: 200, window: 4, sg: 1, epochs: 4 ---> Accuracy: 0.24440455488865204, avg_distance: 0.30131969365024586
Modelo: [8], vector_size: 100, window: 6, sg: 1, epochs: 4 ---> Accuracy: 0.2683569865933696, avg_dista

In [9]:
print(f"Melhores modelos de acordo com a distância de cosseno:")
top_10_models2 = sorted(model_metrics2, key=lambda x: x['avg_distance'])[:10]
for model_info2 in top_10_models2:
    print(f"Model: {model_info2['model_name']}, vector_size: {model_info2['vector_size']}, window: {model_info2['window']}, sg: {model_info2['sg']}, epochs: {model_info2['epochs']} ---> Accuracy: {model_info2['accuracy']}, avg_distance: {model_info2['avg_distance']}")


print(f"\n\nMelhores modelos de acordo com a acurácia:")
top_10_models2 = sorted(model_metrics2, key=lambda x: x['accuracy'], reverse=True)[:10]
for model_info2 in top_10_models2:
    print(f"Model: {model_info2['model_name']}, vector_size: {model_info2['vector_size']}, window: {model_info2['window']}, sg: {model_info2['sg']}, epochs: {model_info2['epochs']} ---> Accuracy: {model_info2['accuracy']}, avg_distance: {model_info2['avg_distance']}")

Melhores modelos de acordo com a distância de cosseno:
Model: word2vec_vs100_win4_sg1_ep4.model, vector_size: 100, window: 4, sg: 1, epochs: 4 ---> Accuracy: 0.2467044370898076, avg_distance: 0.2605604038461488
Model: word2vec_vs100_win6_sg1_ep4.model, vector_size: 100, window: 6, sg: 1, epochs: 4 ---> Accuracy: 0.2683569865933696, avg_distance: 0.2612182565868032
Model: word2vec_vs100_win2_sg1_ep4.model, vector_size: 100, window: 2, sg: 1, epochs: 4 ---> Accuracy: 0.19706063835754753, avg_distance: 0.2629579813795871
Model: word2vec_vs100_win4_sg1_ep8.model, vector_size: 100, window: 4, sg: 1, epochs: 8 ---> Accuracy: 0.3031356930498682, avg_distance: 0.2819718109176635
Model: word2vec_vs100_win6_sg1_ep8.model, vector_size: 100, window: 6, sg: 1, epochs: 8 ---> Accuracy: 0.3125035059179896, avg_distance: 0.2837921193431643
Model: word2vec_vs100_win2_sg1_ep8.model, vector_size: 100, window: 2, sg: 1, epochs: 8 ---> Accuracy: 0.2559039658944298, avg_distance: 0.2915923016449943
Model: w

Dado os 2 modelos os melhores resultados foram:

Distância de cosseno:
---------------------
Model: vector_size: 50, window: 5, sg: 1, epochs: 5 ---> Accuracy: 0.2193863241151063, avg_distance: 0.23655692735678963

Model: vector_size: 100, window: 4, sg: 1, epochs: 4 ---> Accuracy: 0.2467044370898076, avg_distance: 0.2605604038461488


Acurácia:
---------

Model: vector_size: 200, window: 7, sg: 1, epochs: 15 ---> Accuracy: 0.35917428619509734, avg_distance: 0.3641003696024459

Model: vector_size: 200, window: 6, sg: 1, epochs: 16 ---> Accuracy: 0.355247658046783, avg_distance: 0.36692009469933407

Melhores Resultaos:
-------------------

Melhor distância de cosseno: vector_size: 100, window: 4, sg: 1, epochs: 4 ---> avg_distance: 0.2605604038461488

Melhor acurácia: vector_size: 200, window: 7, sg: 1, epochs: 15 ---> Accuracy: 0.35917428619509734